In [1]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
import pickle

In [2]:
import sys
import os

# Ensure Python finds config.py
sys.path.append(os.path.abspath(".."))  # Moves up one level to find config.py

In [3]:
import pandas as pd
import config
from config import PROCESSED_DATA_DIR  # Import the global path setup


# Load data using the relative path from config.py
file_path = os.path.join(PROCESSED_DATA_DIR, "varmax_train_data.csv")  
df = pd.read_csv(file_path, index_col="date", parse_dates=True)



In [4]:
# Ensure datetime format & monthly frequency
df.index = pd.to_datetime(df.index)
df = df.asfreq("ME")

# Define endogenous variables (macro variables excluding FFR)
endog_vars = df.drop(columns=["FEDFUNDS"])

# Define exogenous variable (FFR)
exog_vars = df[["FEDFUNDS"]]



In [5]:
#  8. Select Optimal Lag Order Using VAR
var_model = VAR(endog_vars)  # Use standard VAR for lag selection
lag_selection = var_model.select_order(maxlags=12)  # Try up to 12 lags


In [6]:
# Extract best lag based on AIC (it is already a single value)
optimal_lag = lag_selection.aic
print(f"Optimal lag selected based on AIC: {optimal_lag}")


Optimal lag selected based on AIC: 4


In [7]:
from config import MODELS_DIR
import os
import pickle
from statsmodels.tsa.statespace.varmax import VARMAX

# Ensure models directory exists
os.makedirs(MODELS_DIR, exist_ok=True)

# Define model path
varmax_model_path = os.path.join(MODELS_DIR, "varmax_model.pkl")

# 9. Fit VARMAX Model with Optimal Lag
print("Training VARMAX Model...")

# Shift exogenous variables to match lag structure
exog_vars_lagged = exog_vars.iloc[optimal_lag:]  # Drop initial lagged rows
endog_vars_lagged = endog_vars.iloc[optimal_lag:]  # Drop initial lagged rows

# Fit VARMAX Model
varmax_model = VARMAX(endog_vars_lagged, exog=exog_vars_lagged, order=(optimal_lag, 0))
varmax_results = varmax_model.fit(disp=False)

print("VARMAX Model Training Complete!")
print(varmax_results.summary())

# 10. Save the Trained Model
with open(varmax_model_path, "wb") as f:
    pickle.dump(varmax_results, f)

print(f"VARMAX Model Saved: {varmax_model_path}")


Training VARMAX Model...


/Applications/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


VARMAX Model Training Complete!
                                                Statespace Model Results                                               
Dep. Variable:     ['CPIAUCSL', 'UNRATE', 'INDPRO', 'M2SL', 'GDPC1', 'GDPPOT']   No. Observations:                  166
Model:                                                                 VARX(4)   Log Likelihood               -1617.618
                                                                   + intercept   AIC                           3589.237
Date:                                                         Wed, 19 Mar 2025   BIC                           4140.058
Time:                                                                 18:38:43   HQIC                          3812.819
Sample:                                                             02-28-2005                                         
                                                                  - 11-30-2018                                         
Covarian